<a href="https://colab.research.google.com/github/quanndm/kltn/blob/dev/notebooks/liver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Init - import library - download data

In [1]:
!rm -rf kltn
!rm -rf sample_data
!git clone -b dev https://github.com/quanndm/kltn.git

Cloning into 'kltn'...
remote: Enumerating objects: 1319, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 1319 (delta 69), reused 49 (delta 26), pack-reused 1212 (from 2)
Receiving objects: 100% (1319/1319), 13.44 MiB | 8.06 MiB/s, done.
Resolving deltas: 100% (857/857), done.


In [2]:
from kltn.init.install_dependencies import install_packages, load_config
install_packages("./kltn")
config = load_config("./kltn/parameters.yaml")

In [3]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive

from kltn import *
from kltn.datasets.get_datasets import get_datasets_lits
from kltn.utils.logger import init_logger
from kltn.utils.train_val_epochs import  trainer_stage1
from kltn.utils.metrics import  DiceMetric, TverskyLossWSigmoid, DiceLossWSigmoid
from kltn.models.model_factory import ModelFactory
from kltn.utils.visualize_results import visualize_results

In [4]:
drive.flush_and_unmount()
drive.mount("/content/drive")

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [5]:
LOGGER = init_logger(config["log_path"])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
from kltn.datasets.prepare_data import prepare_dataset_LiTS
prepare_dataset_LiTS(dir_name=config["source_folder_lits"])

0.10% complete (down: 0.0 kB/s up: 0.0 kB/s peers: 1) downloading LITS17 (https://academictorrents.com/announce.php)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (1)
LITS17 (https://ipv6.academictorrents.com/announce.php)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (1)
LITS17 (https://academictorrents.com/announce.php)[172.28.0.12:6881] v1 unspecified system error "" (1)
LITS17 (https://ipv6.academictorrents.com/announce.php)[172.28.0.12:6881] v1 unspecified system error "" (1)
0.10% complete (down: 3447.7 kB/s up: 92.2 kB/s peers: 4) downloading LITS17 (udp://tracker.opentrackr.org:1337/announce)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (1)
0.41% complete (down: 3917.6 kB/s up: 118.2 kB/s peers: 9) downloading LITS17 (https://academictorrents.com/announce.php)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (2)
LITS17 (https://ipv6.academictorrents.com/announce.php)[127.0.0.1:6881] v1 skipping tracker announce (unre

## Load data

In [7]:
full_train_dataset, val_dataset = get_datasets_lits(source_folder=config["source_folder_lits"], seed=123, fold_number=2, mode="liver")
print(len(full_train_dataset), len(val_dataset))

105 26


In [8]:
train_loader = torch.utils.data.DataLoader(full_train_dataset, batch_size=1, shuffle=True,
                                           num_workers=1, pin_memory=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False,
                                         pin_memory=True, num_workers=1)

## Training

In [9]:
model = ModelFactory.get_model("unet3d_resnextcot",in_channels=1, n_classes=1, n_channels=16).to(device)

In [10]:
criterion = DiceLossWSigmoid().to(device)

dice_acc = DiceMetric(include_background=False, reduction='mean_batch', get_not_nans=True)

optimizer = torch.optim.AdamW(model.parameters(), lr=float(config["learning_rate"]), weight_decay=float(config["weight_decay"]))
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=int(config["max_epochs"]), eta_min = float(config["eta_min"]))

In [11]:
LOGGER.info("[TRAINER] Start TRAIN process...")

(
    val_acc_max,
    best_epoch,
    dices_liver,
    loss_epochs,
    trains_epoch,
    ious_liver,
    precisions_liver,
    recalls_liver,
    time_tmp
) = trainer_stage1(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    loss_func=criterion,
    acc_func=dice_acc,
    scheduler=scheduler,
    batch_size=config["batch_size"],
    max_epochs = config["max_epochs"],
    start_epoch = config["start_epoch"],
    val_every=config["val_every"],
    path_save_model=config["path_save_model_state"],
    logger=LOGGER,
    save_model=True,
    post_fix="liver"
)

[TRAINER] Start TRAIN process...
INFO:kltn.utils.logger:[TRAINER] Start TRAIN process...

==============================Training epoch 1==============================
INFO:kltn.utils.logger:
==============================Training epoch 1==============================


Epoch 1/100 1/105 loss: 0.6717 time 5.65s
Epoch 1/100 2/105 loss: 0.5836 time 0.68s
Epoch 1/100 3/105 loss: 0.5718 time 0.63s
Epoch 1/100 4/105 loss: 0.5457 time 4.15s
Epoch 1/100 5/105 loss: 0.5354 time 7.42s
Epoch 1/100 6/105 loss: 0.5471 time 4.67s
Epoch 1/100 7/105 loss: 0.5136 time 1.09s
Epoch 1/100 8/105 loss: 0.4851 time 5.44s
Epoch 1/100 9/105 loss: 0.4750 time 4.43s
Epoch 1/100 10/105 loss: 0.4637 time 1.45s
Epoch 1/100 11/105 loss: 0.4437 time 4.97s
Epoch 1/100 12/105 loss: 0.4285 time 4.34s
Epoch 1/100 13/105 loss: 0.4228 time 0.83s
Epoch 1/100 14/105 loss: 0.4221 time 1.10s
Epoch 1/100 15/105 loss: 0.4115 time 2.03s
Epoch 1/100 16/105 loss: 0.4032 time 5.31s
Epoch 1/100 17/105 loss: 0.4146 time 1.88s
Epoch 1/100 18/105 loss: 0.4263 time 6.82s
Epoch 1/100 19/105 loss: 0.4318 time 2.03s
Epoch 1/100 20/105 loss: 0.4221 time 5.23s
Epoch 1/100 21/105 loss: 0.4178 time 7.07s
Epoch 1/100 22/105 loss: 0.4158 time 5.50s
Epoch 1/100 23/105 loss: 0.4122 time 2.35s
Epoch 1/100 24/105 l

Final training epochs: 1/100 ---[loss: 0.2296] ---[time 391.15s]
INFO:kltn.utils.logger:Final training epochs: 1/100 ---[loss: 0.2296] ---[time 391.15s]

********************Epoch 1 Validation********************
INFO:kltn.utils.logger:
********************Epoch 1 Validation********************


Epoch 1/100 105/105 loss: 0.2296 time 5.04s


Val 1/100 1/26, Dice_Liver: 0.847300, time 4.47s
INFO:kltn.utils.logger:Val 1/100 1/26, Dice_Liver: 0.847300, time 4.47s
Val 1/100 2/26, Dice_Liver: 0.910175, time 0.18s
INFO:kltn.utils.logger:Val 1/100 2/26, Dice_Liver: 0.910175, time 0.18s
Val 1/100 3/26, Dice_Liver: 0.898597, time 4.59s
INFO:kltn.utils.logger:Val 1/100 3/26, Dice_Liver: 0.898597, time 4.59s
Val 1/100 4/26, Dice_Liver: 0.872643, time 6.63s
INFO:kltn.utils.logger:Val 1/100 4/26, Dice_Liver: 0.872643, time 6.63s
Val 1/100 5/26, Dice_Liver: 0.892710, time 0.68s
INFO:kltn.utils.logger:Val 1/100 5/26, Dice_Liver: 0.892710, time 0.68s
Val 1/100 6/26, Dice_Liver: 0.875914, time 4.90s
INFO:kltn.utils.logger:Val 1/100 6/26, Dice_Liver: 0.875914, time 4.90s
Val 1/100 7/26, Dice_Liver: 0.922641, time 5.08s
INFO:kltn.utils.logger:Val 1/100 7/26, Dice_Liver: 0.922641, time 5.08s
Val 1/100 8/26, Dice_Liver: 0.821192, time 3.96s
INFO:kltn.utils.logger:Val 1/100 8/26, Dice_Liver: 0.821192, time 3.96s
Val 1/100 9/26, Dice_Liver: 0.90

...

Val 99/100 1/26, Dice_Liver: 0.943703, time 1.36s
INFO:kltn.utils.logger:Val 99/100 1/26, Dice_Liver: 0.943703, time 1.36s
Val 99/100 2/26, Dice_Liver: 0.964954, time 0.79s
INFO:kltn.utils.logger:Val 99/100 2/26, Dice_Liver: 0.964954, time 0.79s
Val 99/100 3/26, Dice_Liver: 0.973096, time 7.24s
INFO:kltn.utils.logger:Val 99/100 3/26, Dice_Liver: 0.973096, time 7.24s
Val 99/100 4/26, Dice_Liver: 0.966991, time 6.74s
INFO:kltn.utils.logger:Val 99/100 4/26, Dice_Liver: 0.966991, time 6.74s
Val 99/100 5/26, Dice_Liver: 0.965202, time 0.76s
INFO:kltn.utils.logger:Val 99/100 5/26, Dice_Liver: 0.965202, time 0.76s
Val 99/100 6/26, Dice_Liver: 0.973444, time 6.03s
INFO:kltn.utils.logger:Val 99/100 6/26, Dice_Liver: 0.973444, time 6.03s
Val 99/100 7/26, Dice_Liver: 0.970318, time 5.26s
INFO:kltn.utils.logger:Val 99/100 7/26, Dice_Liver: 0.970318, time 5.26s
Val 99/100 8/26, Dice_Liver: 0.966579, time 4.25s
INFO:kltn.utils.logger:Val 99/100 8/26, Dice_Liver: 0.966579, time 4.25s
Val 99/100 9/26,

Epoch 100/100 1/105 loss: 0.0300 time 3.26s
Epoch 100/100 2/105 loss: 0.0297 time 6.07s
Epoch 100/100 3/105 loss: 0.0276 time 5.58s
Epoch 100/100 4/105 loss: 0.0308 time 2.64s
Epoch 100/100 5/105 loss: 0.0296 time 5.54s
Epoch 100/100 6/105 loss: 0.0283 time 4.76s
Epoch 100/100 7/105 loss: 0.0280 time 5.93s
Epoch 100/100 8/105 loss: 0.0284 time 3.17s
Epoch 100/100 9/105 loss: 0.0276 time 6.11s
Epoch 100/100 10/105 loss: 0.0273 time 6.02s
Epoch 100/100 11/105 loss: 0.0278 time 2.10s
Epoch 100/100 12/105 loss: 0.0277 time 0.69s
Epoch 100/100 13/105 loss: 0.0283 time 1.21s
Epoch 100/100 14/105 loss: 0.0286 time 0.90s
Epoch 100/100 15/105 loss: 0.0281 time 6.78s
Epoch 100/100 16/105 loss: 0.0281 time 5.00s
Epoch 100/100 17/105 loss: 0.0281 time 1.60s
Epoch 100/100 18/105 loss: 0.0291 time 1.29s
Epoch 100/100 19/105 loss: 0.0287 time 8.60s
Epoch 100/100 20/105 loss: 0.0284 time 4.36s
Epoch 100/100 21/105 loss: 0.0288 time 6.18s
Epoch 100/100 22/105 loss: 0.0294 time 2.52s
Epoch 100/100 23/10

Final training epochs: 100/100 ---[loss: 0.0294] ---[time 402.78s]
INFO:kltn.utils.logger:Final training epochs: 100/100 ---[loss: 0.0294] ---[time 402.78s]

********************Epoch 100 Validation********************
INFO:kltn.utils.logger:
********************Epoch 100 Validation********************


Epoch 100/100 105/105 loss: 0.0294 time 1.23s


Val 100/100 1/26, Dice_Liver: 0.943825, time 1.33s
INFO:kltn.utils.logger:Val 100/100 1/26, Dice_Liver: 0.943825, time 1.33s
Val 100/100 2/26, Dice_Liver: 0.965074, time 0.92s
INFO:kltn.utils.logger:Val 100/100 2/26, Dice_Liver: 0.965074, time 0.92s
Val 100/100 3/26, Dice_Liver: 0.972959, time 6.26s
INFO:kltn.utils.logger:Val 100/100 3/26, Dice_Liver: 0.972959, time 6.26s
Val 100/100 4/26, Dice_Liver: 0.966962, time 7.08s
INFO:kltn.utils.logger:Val 100/100 4/26, Dice_Liver: 0.966962, time 7.08s
Val 100/100 5/26, Dice_Liver: 0.964514, time 0.89s
INFO:kltn.utils.logger:Val 100/100 5/26, Dice_Liver: 0.964514, time 0.89s
Val 100/100 6/26, Dice_Liver: 0.973033, time 5.19s
INFO:kltn.utils.logger:Val 100/100 6/26, Dice_Liver: 0.973033, time 5.19s
Val 100/100 7/26, Dice_Liver: 0.971460, time 5.49s
INFO:kltn.utils.logger:Val 100/100 7/26, Dice_Liver: 0.971460, time 5.49s
Val 100/100 8/26, Dice_Liver: 0.966631, time 4.63s
INFO:kltn.utils.logger:Val 100/100 8/26, Dice_Liver: 0.966631, time 4.63s


In [12]:
import json
path_save_result = f"/content/drive/MyDrive/KLTN/code/result_model_{model.__class__.__name__}_liver.json"

results = {
    "dice_liver": [float(x) for x in np.array(dices_liver, dtype=np.float32)],
    "loss": [float(x) for x in np.array(loss_epochs, dtype= np.float32)],
    "iou_liver": [float(x) for x in np.array(ious_liver, dtype=np.float32)],
    "precision_liver": [float(x) for x in np.array(precisions_liver, dtype=np.float32)],
    "recall_liver": [float(x) for x in np.array(recalls_liver, dtype=np.float32)],
    "best_epoch": best_epoch,
    "time_train": time_tmp
}
with open(path_save_result, "w") as f:
    json.dump(results, f, indent=4)